# Creditworthiness (Project 4)

### Problem setting

A bank wants to predict the creditworthiness of its customers. Based on the customer records, the credit history, etc., a customer should be classified as creditworthy or unworthy of credit. It is five times more 'expensive' for the bank to rate a customer who is unworthy of credit as creditworthy than vice versa. In addition, not all information is available for all customers. For 1,000 representatively selected customers, the creditworthiness is known. For these customers the following data has been collected. (Features for which not all values are known are marked with the addition "incomplete".)

### Importing Python packages

Importing all the python packages we are going to use:

In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

plt.style.use("seaborn") #setting plot style

### Reading in the data

First of all we need to import the data from the given file 'kredit.dat' and create a data frame with it.

In [58]:
#read flash.dat to a list of lists
datContent = [i.strip().split() for i in open("./kredit.dat").readlines()]

#write it as a new CSV file
with open("./kredit.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(datContent)

#naming the labels of the columns
columns = ['Status of existing checking account','Duration in month','Credit history','Purpose','Credit amount','Savings account/bonds','Present employment since','Installment rate in percentage of disposable income',
'Personal status and sex','Other debtors/guarantors','Present residence since','Porperty','Age in years','Other installment plans','Housing','Number of existing credits at this bank','Job','Number of people being liable to provide maintenance for',
'Telephone','Foreign worker','Creditworthy']

#creating the dataframe
df = pd.read_csv('./kredit.csv',names=columns)
df.head()

,Status of existing checking account,Duration in month,Credit history,Purpose,Credit amount,Savings account/bonds,Present employment since,Installment rate in percentage of disposable income,Personal status and sex,Other debtors/guarantors,...,Porperty,Age in years,Other installment plans,Housing,Number of existing credits at this bank,Job,Number of people being liable to provide maintenance for,Telephone,Foreign worker,Creditworthy
0,A14,36,A32,?,2299,A63,?,4,A93,A101,...,A123,39,A143,A152,1,A173,1,A191,?,1
1,A12,18,A32,A46,1239,A65,A73,4,A93,A101,...,A124,61,A143,A153,1,?,1,A191,A201,1
2,A13,24,A32,A40,947,A61,A74,4,A93,A101,...,A124,38,A141,A153,1,?,2,A191,?,2
3,A14,15,A33,A43,1478,A61,A73,4,A94,A101,...,A121,33,A141,A152,2,A173,1,A191,A201,1
4,A14,24,A32,A40,1525,A64,A74,4,A92,A101,...,A123,34,A143,A152,1,A173,2,A192,A201,1


### Preprocessing the data

With our now build data frame we can move forward and handle the missing values. For that we need to transform some of the attributes, since a lot of them are devided into classes (like A32, A33,...). To train a model that can make a prediction on our missing values we need vectors with only numerical values. Therefore we create new features for our data like that: A30=0, A31=0, A32=1, A33=0, A34=0 (for the "Credit history" of the client being A32). We need to apply that process for every non numerical feature. Lets take a look at which attributes we need to transform.


In [59]:
print(df.dtypes) #display the data type of each column

Status of existing checking account                         object
Duration in month                                            int64
Credit history                                              object
Purpose                                                     object
Credit amount                                                int64
Savings account/bonds                                       object
Present employment since                                    object
Installment rate in percentage of disposable income          int64
Personal status and sex                                     object
Other debtors/guarantors                                    object
Present residence since                                      int64
Porperty                                                    object
Age in years                                                 int64
Other installment plans                                     object
Housing                                                     ob

As we can see the features: "Status of existing checking account", "Credit history", "Purpose", "Savings account/bonds", "Present employment since", "Personal status and sex", "Other debtors/guarantors", "Porperty", "Other installment plans", "Housing", "Job", "Telephone" and "Foreign worker" are all non numerical features, which we have to transform. Taking a closer look at the data we can aswell see that the attribute "Telephone" only has two possible values (A191 and A192), which means we don't need to create a vector for it, but can transform the data to a binary code within the column (place a 0 for A191 and a 1 for A192). 

In [60]:
style = OneHotEncoder()
non_numerics = df.select_dtypes(include='object')
non_numerics = non_numerics.drop(['Purpose','Present employment since','Job','Foreign worker'],axis=1) #excluding these columns since there are values missing and therefore not transformable
non_numerics = non_numerics.drop('Telephone',axis=1) #only has two classes, which means we can transform it within the column to 0 for A191 and 1 for A192
df.loc[df['Telephone'] == 'A191', 'Telephone'] = 0.0
df.loc[df['Telephone'] == 'A192', 'Telephone'] = 1.0
df['Telephone'] = df['Telephone'].astype('int64')
for i in non_numerics.columns.tolist():
    transformation = style.fit_transform(df[[i]]) #transform column i
    df = df.join(pd.DataFrame(transformation.toarray(), columns=style.categories_[0])) #add new categories (of transformation) to our dataframe
    df = df.drop(i, axis=1) #dropping old column since we transformed its information
df.head()

,Duration in month,Purpose,Credit amount,Present employment since,Installment rate in percentage of disposable income,Present residence since,Age in years,Number of existing credits at this bank,Job,Number of people being liable to provide maintenance for,...,A121,A122,A123,A124,A141,A142,A143,A151,A152,A153
0,36,?,2299,?,4,4,39,1,A173,1,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,18,A46,1239,A73,4,4,61,1,?,1,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
2,24,A40,947,A74,4,3,38,1,?,2,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
3,15,A43,1478,A73,4,3,33,2,A173,1,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,24,A40,1525,A74,4,3,34,1,A173,2,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


By taking a look at the numerical values that we didn't have to transform, we can see that some attributes have different ranges they are varying in. To make sure that the attributes with higher ranges don't have an bigger impact on our decision function we perform a Z-Score normalization on the numerical attributes that don't have a range from 0 to 1: "Duration in month", "Credit amount" and "Age in years", "Installment rate in percentage of disposable income", "Present residence since", "Number of existing credits at this bank" and "Number of people being liable to provide maintenance for".

In [61]:
def z_score(column):
    mean = np.mean(column)
    std = np.std(column)
    result = []
    for i in column:
        result.append((i - mean)/std)
    return np.array(result)

df['Duration in month'] = z_score(df['Duration in month'])
df['Credit amount'] = z_score(df['Credit amount'])
df['Age in years'] = z_score(df['Age in years'])
df['Installment rate in percentage of disposable income'] = z_score(df['Installment rate in percentage of disposable income'])
df['Present residence since'] = z_score(df['Present residence since'])
df['Number of existing credits at this bank'] = z_score(df['Number of existing credits at this bank'])
df['Number of people being liable to provide maintenance for'] = z_score(df['Number of people being liable to provide maintenance for'])

df.head()

,Duration in month,Purpose,Credit amount,Present employment since,Installment rate in percentage of disposable income,Present residence since,Age in years,Number of existing credits at this bank,Job,Number of people being liable to provide maintenance for,...,A121,A122,A123,A124,A141,A142,A143,A151,A152,A153
0,1.252574,?,-0.344610,?,0.918477,1.046987,0.303788,-0.704926,A173,-0.428290,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,-0.240857,A46,-0.720320,A73,0.918477,1.046987,2.238742,-0.704926,?,-0.428290,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
2,0.256953,A40,-0.823818,A74,0.918477,0.140505,0.215835,-0.704926,?,2.334869,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
3,-0.489762,A43,-0.635608,A73,0.918477,0.140505,-0.223927,1.027079,A173,-0.428290,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.256953,A40,-0.618950,A74,0.918477,0.140505,-0.135974,-0.704926,A173,2.334869,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


With the next step we are going to rearrange the columns of the data frame that our features with the missing values are the first four columns. This has no other purpose than giving us a better overview of the data.

In [62]:
old_order = df.columns.tolist()
new_order = [old_order[1]] + [old_order[3]] + [old_order[8]] + [old_order[11]] + [old_order[0]] + [old_order[2]] + old_order[4:8] + old_order[9:11] + old_order[12:]
df = df[new_order]
df.head()

,Purpose,Present employment since,Job,Foreign worker,Duration in month,Credit amount,Installment rate in percentage of disposable income,Present residence since,Age in years,Number of existing credits at this bank,...,A121,A122,A123,A124,A141,A142,A143,A151,A152,A153
0,?,?,A173,?,1.252574,-0.344610,0.918477,1.046987,0.303788,-0.704926,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,A46,A73,?,A201,-0.240857,-0.720320,0.918477,1.046987,2.238742,-0.704926,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
2,A40,A74,?,?,0.256953,-0.823818,0.918477,0.140505,0.215835,-0.704926,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
3,A43,A73,A173,A201,-0.489762,-0.635608,0.918477,0.140505,-0.223927,1.027079,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,A40,A74,A173,A201,0.256953,-0.618950,0.918477,0.140505,-0.135974,-0.704926,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


Also we transform the attribute "Creditworthy" since it is encoded with 2 for "not creditworthy" and 1 for "creditworthy". We change all 2 to 0, so our data follows the same pattern as we have encoded the categorical values with 0's and 1's aswell.

In [63]:
df.loc[df['Creditworthy'] == 2, 'Creditworthy'] = 0
df['Creditworthy'].head()

0    1
1    1
2    0
3    1
4    1
Name: Creditworthy, dtype: int64

### Handling missing values

First lets take a closer look at the attributes. We can see that the category "Foreign worker" only has two possible values either "yes" or "no". Lets start with this attribute since its going to be the simplest. But first we transorm the values "A202" and "A201" to 0 (no) and 1 (yes).

In [64]:
df.loc[df['Foreign worker'] == 'A202', 'Foreign worker'] = 0
df.loc[df['Foreign worker'] == 'A201', 'Foreign worker'] = 1

data_points = len(df.loc[df['Foreign worker'] != '?', 'Foreign worker'])
fw = len(df.loc[df['Foreign worker'] == 1, 'Foreign worker'])
ratio = fw/data_points



0.9578125
